# Detecting Faulty $f$-DP Claim for the Gaussian Mechanism 

## Description

This notebook demonstrates how to use the auditor API to audit a claimed tradeoff curve for the Gaussian Mechanism.

### Step 1: Import Packages

In [1]:
import numpy as np
import os
import sys
import time
import logging
import matplotlib.pyplot as plt
from scipy.stats import norm

# Navigate to the parent directory of the project structure
project_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
src_dir = os.path.join(project_dir, 'src')
log_dir = os.path.join(project_dir, 'log')
fig_dir = os.path.join(project_dir, 'fig')
os.makedirs(fig_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
logfile_path = os.path.join(log_dir, 'auditing-Gaussian.log')

# Add the src directory to sys.path
sys.path.append(src_dir)


from analysis.accuracy_analysis import create_plot, knn_baybox_acc_bound_1d

from mech.GaussianDist import *

### Step 2: Instantiate the $f$-DP Auditor for Gaussian Mechanism

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(message)s",
    handlers=[
        logging.FileHandler(logfile_path)
        # logging.StreamHandler()
    ]
)

def claimed_curve(alpha):
    return norm.cdf(norm.ppf(1 - alpha) - 0.2)

kwargs = generate_params(claimed_f = claimed_curve, num_train_samples=1000000, num_test_samples=1000000)
auditor = GaussianAuditor(kwargs)

### Step 3: Auditing and Report

In [ ]:
output = auditor.build()

In [ ]:
output["Report"] 

In [ ]:
file_name = os.path.join(fig_dir, 'faulty_gaussian_108.png')
omega_8=knn_baybox_acc_bound_1d(10**8, kwargs["gamma"])

create_plot(omega_8, output["estimated_alpha"], output["estimated_beta"], np.linspace(0,1,500), claimed_curve, output["scan_alpha"], output["scan_beta"], output["critical_alpha"], output["critical_beta"], file_name)